In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB0
from keras_squeeze_excite_network.se_resnext import SEResNextImageNet as SEResNext

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            #imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [10]:
def getMultiBased():
    model =  DenseNet169(weights="imagenet", include_top=False, input_shape=(*size, 3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=8000)
del df

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-dense-epoch{epoch:04}.h5")

In [13]:
batch_size=128
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size, size=size)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size, size=size)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
#model.summary()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=64, use_multiprocessing=True)
model.save("multiDenseNet.h5")

compiled
Epoch 1/80
1413/1413 [==============================] - 931s 659ms/step - loss: 2.3257 - dense_1_loss: 3.5122 - dense_2_loss: 1.4130 - dense_3_loss: 0.8655 - dense_1_acc: 0.3515 - dense_2_acc: 0.6486 - dense_3_acc: 0.7521 - val_loss: 0.7170 - val_dense_1_loss: 1.0237 - val_dense_2_loss: 0.4648 - val_dense_3_loss: 0.3437 - val_dense_1_acc: 0.7893 - val_dense_2_acc: 0.9046 - val_dense_3_acc: 0.8679
Epoch 2/80
1413/1413 [==============================] - 746s 528ms/step - loss: 1.7396 - dense_1_loss: 2.5599 - dense_2_loss: 1.1431 - dense_3_loss: 0.6957 - dense_1_acc: 0.6137 - dense_2_acc: 0.7309 - dense_3_acc: 0.8140 - val_loss: 0.5104 - val_dense_1_loss: 0.7673 - val_dense_2_loss: 0.3005 - val_dense_3_loss: 0.2246 - val_dense_1_acc: 0.8507 - val_dense_2_acc: 0.9491 - val_dense_3_acc: 0.9466
Epoch 3/80
1413/1413 [==============================] - 740s 524ms/step - loss: 1.5708 - dense_1_loss: 2.2846 - dense_2_loss: 1.0597 - dense_3_loss: 0.6541 - dense_1_acc: 0.6386 - dense_2_acc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 744s 527ms/step - loss: 1.1253 - dense_1_loss: 1.5487 - dense_2_loss: 0.8459 - dense_3_loss: 0.5578 - dense_1_acc: 0.6714 - dense_2_acc: 0.7658 - dense_3_acc: 0.8289 - val_loss: 0.1984 - val_dense_1_loss: 0.2530 - val_dense_2_loss: 0.0874 - val_dense_3_loss: 0.0921 - val_dense_1_acc: 0.9329 - val_dense_2_acc: 0.9808 - val_dense_3_acc: 0.9750
Epoch 9/80
1413/1413 [==============================] - 742s 525ms/step - loss: 1.0893 - dense_1_loss: 1.4898 - dense_2_loss: 0.8260 - dense_3_loss: 0.5514 - dense_1_acc: 0.6745 - dense_2_acc: 0.7752 - dense_3_acc: 0.8306 - val_loss: 0.1465 - val_dense_1_loss: 0.2476 - val_dense_2_loss: 0.0914 - val_dense_3_loss: 0.0943 - val_dense_1_acc: 0.9329 - val_dense_2_acc: 0.9765 - val_dense_3_acc: 0.9725
Epoch 10/80
1413/1413 [==============================] - 737s 522ms/step - loss: 1.0464 - dense_1_loss: 1.4201 - dense_2_loss: 0.8042 - dense_3_loss: 0.5414 - dense_1_acc: 0.6822 - dense_2_acc: 0.7876 - dense_3_

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 745s 527ms/step - loss: 0.8225 - dense_1_loss: 1.0631 - dense_2_loss: 0.6859 - dense_3_loss: 0.4779 - dense_1_acc: 0.7472 - dense_2_acc: 0.8662 - dense_3_acc: 0.8798 - val_loss: 0.1447 - val_dense_1_loss: 0.2099 - val_dense_2_loss: 0.0820 - val_dense_3_loss: 0.0688 - val_dense_1_acc: 0.9443 - val_dense_2_acc: 0.9821 - val_dense_3_acc: 0.9811
Epoch 26/80
1413/1413 [==============================] - 736s 521ms/step - loss: 0.8165 - dense_1_loss: 1.0527 - dense_2_loss: 0.6822 - dense_3_loss: 0.4784 - dense_1_acc: 0.7501 - dense_2_acc: 0.8686 - dense_3_acc: 0.8789 - val_loss: 0.1259 - val_dense_1_loss: 0.1964 - val_dense_2_loss: 0.0689 - val_dense_3_loss: 0.0652 - val_dense_1_acc: 0.9462 - val_dense_2_acc: 0.9838 - val_dense_3_acc: 0.9818
Epoch 27/80
1413/1413 [==============================] - 744s 527ms/step - loss: 0.8110 - dense_1_loss: 1.0442 - dense_2_loss: 0.6801 - dense_3_loss: 0.4755 - dense_1_acc: 0.7528 - dense_2_acc: 0.8723 - dense_3

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 741s 525ms/step - loss: 0.6873 - dense_1_loss: 0.8491 - dense_2_loss: 0.6225 - dense_3_loss: 0.4284 - dense_1_acc: 0.8156 - dense_2_acc: 0.9143 - dense_3_acc: 0.9209 - val_loss: 0.1155 - val_dense_1_loss: 0.1815 - val_dense_2_loss: 0.0680 - val_dense_3_loss: 0.0545 - val_dense_1_acc: 0.9550 - val_dense_2_acc: 0.9857 - val_dense_3_acc: 0.9866
Epoch 66/80
1413/1413 [==============================] - 740s 524ms/step - loss: 0.6855 - dense_1_loss: 0.8465 - dense_2_loss: 0.6210 - dense_3_loss: 0.4279 - dense_1_acc: 0.8197 - dense_2_acc: 0.9145 - dense_3_acc: 0.9211 - val_loss: 0.1913 - val_dense_1_loss: 0.2415 - val_dense_2_loss: 0.3659 - val_dense_3_loss: 0.3382 - val_dense_1_acc: 0.9511 - val_dense_2_acc: 0.9617 - val_dense_3_acc: 0.9655
Epoch 67/80
 668/1413 [=============>................] - ETA: 5:58 - loss: 0.6797 - dense_1_loss: 0.8371 - dense_2_loss: 0.6192 - dense_3_loss: 0.4253 - dense_1_acc: 0.8199 - dense_2_acc: 0.9132 - dense_3_acc: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1413/1413 [==============================] - 743s 526ms/step - loss: 0.6749 - dense_1_loss: 0.8292 - dense_2_loss: 0.6164 - dense_3_loss: 0.4245 - dense_1_acc: 0.8253 - dense_2_acc: 0.9183 - dense_3_acc: 0.9241 - val_loss: 0.2558 - val_dense_1_loss: 0.3937 - val_dense_2_loss: 1.0273 - val_dense_3_loss: 0.3037 - val_dense_1_acc: 0.9399 - val_dense_2_acc: 0.9322 - val_dense_3_acc: 0.9639
Epoch 74/80
1413/1413 [==============================] - 737s 521ms/step - loss: 0.6734 - dense_1_loss: 0.8271 - dense_2_loss: 0.6162 - dense_3_loss: 0.4231 - dense_1_acc: 0.8260 - dense_2_acc: 0.9197 - dense_3_acc: 0.9260 - val_loss: 0.1876 - val_dense_1_loss: 0.3254 - val_dense_2_loss: 0.6620 - val_dense_3_loss: 0.1248 - val_dense_1_acc: 0.9459 - val_dense_2_acc: 0.9559 - val_dense_3_acc: 0.9806
Epoch 75/80
1102/1413 [======================>.......] - ETA: 2:30 - loss: 0.6704 - dense_1_loss: 0.8233 - dense_2_loss: 0.6135 - dense_3_loss: 0.4216 - dense_1_acc: 0.8282 - dense_2_acc: 0.9186 - dense_3_acc: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

